In [6]:
#tensorflow, keras
import keras
from keras.layers import merge, Conv2D, Input, Reshape, Activation
from keras.models import Model, load_model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Lambda, Flatten, Dense
from keras.utils.vis_utils import plot_model
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
from numpy import genfromtxt
from keras.preprocessing.image import img_to_array as img_to_array
from keras.preprocessing.image import load_img as load_img

In [2]:
#essentials
import numpy as np
import math
import os
import random
import pandas as pd
import tensorflow as tf
import argparse
from keras import callbacks

In [4]:
def ConvNet(input_shape):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # First Block
    #CONV layer
    X = Conv2D(32, (7, 7), strides = (1,1), activation='relu', padding='same')(X_input)
    # MAXPOOL + BatchNorm
    X = MaxPooling2D((2,2), strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Second Block
    #CONV layer
    X = Conv2D(64, (5, 5), strides = (2,2), activation='relu', padding='same')(X)
    # MAXPOOL
    X = MaxPooling2D((2,2), strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Third Block
    X = Conv2D(128, (5,5), strides = (1, 1),activation='relu', padding='same')(X)
    # MAXPOOL
    X = MaxPooling2D(pool_size = 3, strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Top layer
    X = AveragePooling2D(pool_size=(2,2), strides=(2,2))(X)
    X = Conv2D(64, (7,7), strides = (2,2),activation='relu')(X)
    
    # L2 normalization
    X = Lambda(lambda  x: K.l2_normalize(x,axis=-1))(X)

    #Final output layer. First Unit is a sigmoid act(whether seen img is infected/not)
    # next 2 units for identifying type of infection if 1st element is 1. otherwise, don't care.
    
    infected = Conv2D(2, (1, 1), strides = (1,1),activation='softmax')(X)
    infection_type = Conv2D(3, (1, 1), strides = (1,1),activation='softmax')(X)
    
    infected= Reshape((2,))(infected)
    infection_type= Reshape((3,))(infection_type)
    # Create model instance
    
    model = Model(inputs = X_input, outputs = (infected, infection_type))
    
    return model

In [7]:
c= ConvNet((256,256,3))
c.load_weights('checkpointSave.hdf5')

In [4]:
m=100

Xtrain= np.random.randn(m, 256, 256, 3)
y1train = np.random.randn(m, 2)
y2train = np.random.randn(m, 3)


y = [y1train, y2train]


##Using the GPU
with tf.device('/device:GPU:0'):

    c.fit(x= Xtrain, y=y, epochs=2, batch_size=25)

Epoch 1/2
100/100 [==============================] - 13s 134ms/step - loss: -0.1614 - reshape_1_loss: -0.1329 - reshape_2_loss: -0.0285 - reshape_1_acc: 0.4900 - reshape_2_acc: 0.3000
Epoch 2/2
100/100 [==============================] - 0s 4ms/step - loss: -1.2357 - reshape_1_loss: -0.5186 - reshape_2_loss: -0.7172 - reshape_1_acc: 0.8400 - reshape_2_acc: 0.8100


In [5]:
def load_image(image_path, size=256):
    # data augmentation logic such as random rotations can be added here
    return img_to_array(load_img(image_path, target_size=(size, size, 3))) / 255.

In [6]:
class InfectedLeavesSequence(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X, y1, y2, batch_size=512, dim=(256,256), n_channels=3, shuffle=True):
        #Initialization
        self.dim = dim
        self.X= X
        self.y1= y1
        self.y2= y2
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        batched_image_names = [self.X[k] for k in indexes]
        batched_y1 = self.y1[indexes]
        batched_y2 = self.y2[indexes]
        
        # Generate data
        X = self.__data_generation(batched_image_names)
        
        return X, [batched_y1, batched_y2]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, images):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))
        
        # Generate data
        for i, ID in enumerate(images):
            # Store sample
            X[i,:, :, :] = load_image('data_images/Archive/'+ID)

        return X

In [7]:
job_dir = ''

In [8]:
random.seed(0)
from sklearn.model_selection import StratifiedKFold

In [9]:
def prep_data(df_path='infectedLeaves.csv', n_classes1= 2, n_classes2= 3):
    df = pd.read_csv(df_path)
    #resolve priyanka's error.
    df['infectionType']= df['infectionType'].map({0: 0, 1:1, 2:2, 4:3})
    #create categorical quants
    y1= keras.utils.to_categorical(df['infected'], num_classes=n_classes1)
    y2= keras.utils.to_categorical(df['infectionType'], num_classes=n_classes2+1)
    #drop final axis=-1 last dim of y2
    y2= y2[:, :3]
    #final images list names
    X = df['pathName'].values
    return df, X, y1, y2

In [10]:
df, X, y1, y2= prep_data()

In [11]:
def loss(y_true, y_pred, N = 128, beta=128, epsilon=1e-8):
    
    infected, infection_type = y_pred[0], y_pred[1]
    
    # loss for either layers
    infected_loss = tf.losses.softmax_cross_entropy(y_true[0], infected)
    infection_type_loss = tf.losses.softmax_cross_entropy(y_true[1], infection_type)
    
    # compute loss
    loss = 0.25*infected_loss + 0.75*infection_type_loss
    
    return loss

In [14]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cvscores = []

##Setting up the path for saving logs
logs_path = job_dir + 'logs/tensorboard'

##Using the GPU
with tf.device('/device:GPU:0'):

    tf.reset_default_graph()
    
    ## Initializing the model
    model = ConvNet((256, 256, 3))

    ## Compling the model
    model.compile(optimizer = "adam" , loss = loss, metrics = ["accuracy"]);

    ## Printing the modle summary
    print(model.summary())
    
    
    # Loop through the indices the split() method returns
    for index, (train_indices, val_indices) in enumerate(kfold.split(X, df['infectionType'])):
        if (index==1): 
            break
        print("Training on fold " + str(index+1) + "/5...")
        m_train= len(train_indices)
        m_val= len(val_indices)
        X_train, X_val = X[train_indices], X[val_indices]
        y1_train, y1_val = y1[train_indices], y1[val_indices]
        y2_train, y2_val = y2[train_indices], y2[val_indices]
        
        train_seq = InfectedLeavesSequence(X_train, y1_train, y2_train)
        val_seq = InfectedLeavesSequence(X_val, y1_val, y2_val)

        try:
            hist= model.fit_generator(generator=train_seq,
                        validation_data=val_seq,
                        epochs= 40,
                        
                        )
        
            print(hist.history)
        except: 
            #save model
            model.save('model/fold'+str(index+1)+'.h5')    
        
        scores = model.evaluate(X_val, y_val)
        
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
        cvscores.append(scores[1] * 100)
        
        #save model
        model.save('model_fold'+(index+1)+'.h5')
    #final accuracy
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 4736        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 32) 128         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_2 (

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/usr/lib/python3.5/multiprocessing/context.py", line 267, in _Popen
    return Popen(process_obj)
  File "/usr/lib/python3.5/multiprocessing/popen_fork.py", line 20, in __init__
    self._launch(process_obj)
  File "/usr/lib/python3.5/multiprocessing/popen_fork.py", line 67, in _launch
    self.pid = os.fork(

OSError: Unable to create file (unable to open file: name = 'model/fold1.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
#tensorflow, keras
import keras
from keras.layers import merge, Conv2D, Input, Reshape, Activation
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Lambda, Flatten, Dense
from keras.utils.vis_utils import plot_model
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
from numpy import genfromtxt
from keras.preprocessing.image import img_to_array as img_to_array
from keras.preprocessing.image import load_img as load_img

#essentials
import numpy as np
import math
import os
import random
import pandas as pd
import tensorflow as tf
import argparse
from keras import callbacks

def ConvNet(input_shape):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # First Block
    #CONV layer
    X = Conv2D(32, (7, 7), strides = (1,1), activation='relu', padding='same')(X_input)
    # MAXPOOL + BatchNorm
    X = MaxPooling2D((2,2), strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Second Block
    #CONV layer
    X = Conv2D(64, (5, 5), strides = (2,2), activation='relu', padding='same')(X)
    # MAXPOOL
    X = MaxPooling2D((2,2), strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Third Block
    X = Conv2D(128, (5,5), strides = (1, 1),activation='relu', padding='same')(X)
    # MAXPOOL
    X = MaxPooling2D(pool_size = 3, strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Top layer
    X = AveragePooling2D(pool_size=(2,2), strides=(2,2))(X)
    X = Conv2D(64, (7,7), strides = (2,2),activation='relu')(X)
    
    # L2 normalization
    X = Lambda(lambda  x: K.l2_normalize(x,axis=-1))(X)

    #Final output layer. First Unit is a sigmoid act(whether seen img is infected/not)
    # next 2 units for identifying type of infection if 1st element is 1. otherwise, don't care.
    
    infected = Conv2D(2, (1, 1), strides = (1,1),activation='softmax')(X)
    infection_type = Conv2D(3, (1, 1), strides = (1,1),activation='softmax')(X)
    
    infected= Reshape((2,))(infected)
    infection_type= Reshape((3,))(infection_type)
    # Create model instance
    
    model = Model(inputs = X_input, outputs = (infected, infection_type))
    
    return model

m=100

Xtrain= np.random.randn(m, 256, 256, 3)
y1train = np.random.randn(m, 2)
y2train = np.random.randn(m, 3)


y = [y1train, y2train]

c= ConvNet((256,256,3))
c.compile(optimizer = "adam" , loss = "categorical_crossentropy", metrics = ["accuracy"]);

##Using the GPU
with tf.device('/device:GPU:0'):

    c.fit(x= Xtrain, y=y, epochs=2, batch_size=25)

def load_image(image_path, size=256):
    # data augmentation logic such as random rotations can be added here
    return img_to_array(load_img(image_path, target_size=(size, size, 3))) / 255.

class InfectedLeavesSequence(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X, y1, y2, batch_size=512, dim=(256,256), n_channels=3, shuffle=True):
        #Initialization
        self.dim = dim
        self.X= X
        self.y1= y1
        self.y2= y2
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        batched_image_names = [self.X[k] for k in indexes]
        batched_y1 = self.y1[indexes]
        batched_y2 = self.y2[indexes]
        
        # Generate data
        X = self.__data_generation(batched_image_names)
        
        return X, [batched_y1, batched_y2]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, images):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))
        
        # Generate data
        for i, ID in enumerate(images):
            # Store sample
            X[i,:, :, :] = load_image('data_images/Archive/'+ID)

        return X

job_dir = ''

random.seed(0)
from sklearn.model_selection import StratifiedKFold

def prep_data(df_path='infectedLeaves.csv', n_classes1= 2, n_classes2= 3):
    df = pd.read_csv(df_path)
    #resolve priyanka's error.
    df['infectionType']= df['infectionType'].map({0: 0, 1:1, 2:2, 4:3})
    #create categorical quants
    y1= keras.utils.to_categorical(df['infected'], num_classes=n_classes1)
    y2= keras.utils.to_categorical(df['infectionType'], num_classes=n_classes2+1)
    #drop final axis=-1 last dim of y2
    y2= y2[:, :3]
    #final images list names
    X = df['pathName'].values
    return df, X, y1, y2

df, X, y1, y2= prep_data()

def loss(y_true, y_pred, N = 128, beta=128, epsilon=1e-8):
    
    infected, infection_type = y_pred[0], y_pred[1]
    
    # loss for either layers
    infected_loss = tf.losses.softmax_cross_entropy(y_true[0], infected)
    infection_type_loss = tf.losses.softmax_cross_entropy(y_true[1], infection_type)
    
    # compute loss
    loss = 0.25*infected_loss + 0.75*infection_type_loss
    
    return loss

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cvscores = []

##Setting up the path for saving logs
logs_path = job_dir + 'logs/tensorboard'

##Using the GPU
with tf.device('/device:GPU:0'):

    tf.reset_default_graph()
    
    ## Initializing the model
    model = ConvNet((256, 256, 3))

    ## Compling the model
    model.compile(optimizer = "adam" , loss = loss, metrics = ["accuracy"]);

    ## Printing the modle summary
    print(model.summary())
    
    
    # Loop through the indices the split() method returns
    for index, (train_indices, val_indices) in enumerate(kfold.split(X, df['infectionType'])):
        if (index==1): 
            break
        print("Training on fold " + str(index+1) + "/5...")
        m_train= len(train_indices)
        m_val= len(val_indices)
        X_train, X_val = X[train_indices], X[val_indices]
        y1_train, y1_val = y1[train_indices], y1[val_indices]
        y2_train, y2_val = y2[train_indices], y2[val_indices]
        
        train_seq = InfectedLeavesSequence(X_train, y1_train, y2_train)
        val_seq = InfectedLeavesSequence(X_val, y1_val, y2_val)

        try:
            hist= model.fit_generator(generator=train_seq,
                        validation_data=val_seq,
                        epochs= 40,
                        
                        )
        
            print(hist.history)
        except: 
            #save model
            model.save('model/fold'+str(index+1)+'.h5')    
        
        scores = model.evaluate(X_val, y_val)
        
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
        cvscores.append(scores[1] * 100)
        
        #save model
        model.save('model_fold'+(index+1)+'.h5')
    #final accuracy
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))